In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold, cross_val_score
from patsy import dmatrices, dmatrix
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
%matplotlib inline

Data Fields

1.	Field Name	Description
2.	Submitted_Date	Date and time the application was submitted
3.	Case_No	Case number
4.	Program_Designation	Indicates the type of temporary application submitted for processing.  
a.	R = H-1B
b.	A = E-3 Australian
c.	C = H-1B1 Chile
d.	S = H-1B1 Singapore
5.	Employer_Name	Employer's name
6.	Employer_Address_1	Employer's address
7.	Employer_Address_2	Employer's address2
8.	Employer_City	Employer's city
9.	Employer_State	Employer's state
10.	Employer_Postal_Code	Employer's postal code
11.	Nbr_Immigrants	Number of job openings
12.	Begin_Date	Proposed begin date
13.	End_Date	Proposed end date
14.	Job_Title	Job title
15.	Dol_Decision_Date	Date certified or denied
16.	Certified_Begin_Date	Certification start date
17.	Certified_End_Date	Certification end date
18.	Occupation_Code	Three digit occupational group-see appendix 1 of ETA Form 9035 for the complete list
http://www.foreignlaborcert.doleta.gov/pdf/eta_form_9035cp_appendix_i_121908.pdf 
19.	Case_Status	Approval status - certified or denied
20.	Wage_Rate_From_1	Employer's proposed wage rate
21.	Wage_Rate_Per_1	Unit of pay for proposed wage rate
22.	Wage_Rate_To_1	Maximum proposed wage rate
23.	Part_Time_1	Y = Part time; N = Full time position
24.	Work_City_1	Work city (location of the job opening)
25.	Work_State_1	Work state (location of the job opening)
26.	Prevailing_Wage_1	Prevailing wage rate
27.	Prevailing_Wage_Source_1	Collective bargining; SESA; Other
28.	Year_Source_Published_1	Year that the prevailing wage data was published
29.	Other_Wage_Source_1	Description of the Other wage source (online wage library, OES, employer provided survey, etc.)
30.	Wage_Rate_From_2	Employer's proposed wage rate - second location
31.	Wage_Rate_Per_2	Unit of pay for proposed wage - second location
32.	Wage_Rate_To_2	Maximum proposed wage rate - second location
33.	Part_Time_2	Y = Part time; N = Full time position
34.	Work_City_2	Work city  - second location
35.	Work_State_2	Work state - second location
36.	Prevailing_Wage_2	Prevailing wage rate - second location
37.	Prevailing_Wage_Source_2	Collective bargining; SESA; Other
38.	Year_Source_Published_2	Year that the prevailing wage data was published
39.	Other_Wage_Source_2	         Description of the Other wage source (online wage library, OES, employer provided survey, etc.)


Data is pulled from the following link: http://www.flcdatacenter.com/caseh1b.aspx

In [3]:
data = pd.read_csv('EFILE_FY2007.txt' , delimiter = ",")

IOError: File EFILE_FY2007.txt does not exist

In [ ]:
data.sample()

In [ ]:
data.dtypes

In [ ]:
data.info()

In [ ]:
data['Case_Status'].describe() 

In [ ]:
##we need to factorize the Case_Status column

pd.factorize(data['Case_Status'])
#0= Certified
#1= Denied
#2= Hold
#3= Pending

In [ ]:
label_key = pd.factorize(data['Case_Status'])[1]
data['Status'] = pd.factorize(data['Case_Status'])[0]

In [ ]:
#does this data matter and can we fill in the na's
data.Other_Wage_Source_2.describe()

In [ ]:
data.Year_Source_Published_2.describe()
# Looks like it's only 33705 out of 400,000 rows

In [ ]:
data['Case_Status'].value_counts()

In [ ]:
data.sample(10)

In [ ]:
#drop unneccesary Features
df = data[data.Program_Designation =='R'].drop(['Case Number', 'Address_1', 'Address_2', 'DOL_Decision_Date', 'Certified_Begin_Date', 'Certified_End_Date', 'Begin_Date', 'End_Date', 'Other_Wage_Source_2', 'Year_Source_Published_2' , 'Prevailing_Wage_Source_2', 'Work_State_2', 'Part_Time_2', 'Wage_Rate_To_2', 'Wage_Rate_From_2' ,'Work_City_2', 'Prevailing_Wage_2','Wage_Rate_Per_2', 'Address_2','Wage_Rate_To_1' ], 1)

In [ ]:
df.info()

In [ ]:
df.Program_Designation.describe()
#check to see if only R values are in the table

In [ ]:
df.head(10)

In [ ]:
df.isnull().any()

In [ ]:
df.Occupation_Code=data.Occupation_Code

In [ ]:
df.Occupation_Code.unique()
#looks like we have dates in the occupation code

In [ ]:
#group occupation code then compare with Job Title
df.groupby(df.Occupation_Code).Job_Title.unique()

In [ ]:
##take a look at the dates in the occupation code
df[df['Occupation_Code']=='8/31/2010']

In [ ]:
df2 = df.drop(df[df.Occupation_Code >999].index)
## removes the rows that have the dates

In [ ]:
df2['Prevailing_Wage_1'].isnull().sum()

In [ ]:
df3=df2
#let's see if we can get the media wage for each Occ. Code

In [ ]:
df3['Prevailing_Wage_1'] = df3['Prevailing_Wage_1'].dropna()

In [ ]:
df3['Prevailing_Wage_1'].unique()

In [ ]:
df3['Prevailing_Wage_1'] = df3['Prevailing_Wage_1'].str.lstrip('$')
## remove $ signs

In [ ]:
df3['Prevailing_Wage_1']= df3['Prevailing_Wage_1'].astype(float)


In [ ]:
OCMedian= df3.groupby('Occupation_Code').Prevailing_Wage_1.median().reset_index(inplace= False)
##Find the Media of each Occupation Code

In [ ]:
OCMedian.reset_index(inplace= False)

In [ ]:
df4 = df2
##create a new data set so we can bring in the Median

In [ ]:
df4 = pd.merge(df2, OCMedian, on = 'Occupation_Code', how = 'left')

In [ ]:
list(df4.columns.values)
#check how the columns are listed

In [ ]:
cols = ['Submitted_Date', 'Employer_Name', 'Program_Designation', 'State','Zip_Code','Nbr_Immigrants','Job_Title', 'Occupation_Code','Case_Status','Wage_Rate_From_1','Wage_Rate_Per_1','Part_Time_1','Work_City_1','Work_State_1','Prevailing_Wage_1_x', 'Prevailing_Wage_1_y','Prevailing_Wage_Source_1','Year_Source_Published_1','Other_Wage_Source_1','Withdrawn','Status'
 ] #reorganize the columns so we can back fill

In [ ]:
df4 = df4[cols]

In [ ]:
df4.head()

In [ ]:
df4['Prevailing_Wage_1_x'].fillna(method = 'backfill')
#back fill to fill the na's withe the media

In [ ]:
df4.columns

In [ ]:
df4=df4.drop(['Prevailing_Wage_1_y'], 1)
#drop feature where we backfilled from

In [ ]:
df4

In [ ]:
df4.Occupation_Code.head()

In [ ]:
df4.Wage_Rate_From_1.describe()

In [ ]:
df4.Prevailing_Wage_1_x.describe()

In [ ]:
df4['Wage_Rate_From_1'] = df4['Wage_Rate_From_1'].map(lambda x: x.lstrip('$'))
##remove the dollar sign

In [ ]:
df4['Wage_Rate_From_1'] = df4['Wage_Rate_From_1'].astype(float)

In [ ]:
df4['Wage_Rate_From_1'].describe()

In [ ]:
df4.head()

In [ ]:
df4['Prevailing_Wage_1_x'] = df4['Prevailing_Wage_1_x'].astype(float)

In [ ]:
df4['Wage_Rate_From_1'] = df4['Wage_Rate_From_1'].astype(float)

In [ ]:
df4.describe()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [ ]:
df4.describe()

In [ ]:
df4.isnull().any()

In [ ]:
df4.Case_Status.dropna() 
#drop NA rows for Case_status

In [ ]:
# now let's fill in all of our null values or maybe we can just drop them
df4.Occupation_Code.isnull().sum()

In [ ]:
df4['Occupation_Code'].isnull().sum()

In [ ]:
df4.hist(figsize=(10,10))

In [ ]:
df4['Case_Status'].value_counts()


In [ ]:
df4['Employer_Name'].value_counts().sum()

In [ ]:
df4.columns

In [ ]:
df5= df4[df4['Status'] ==0]
#because mostly everyone is accepted into the program let's see if we can 
#find any kind of patterns what kind of people are consuming the most Visa's

In [ ]:
df5

In [ ]:
formula = 'C(State)  +  Wage_Rate_From_1 + Prevailing_Wage_1_x + Status + Nbr_Immigrants'

In [ ]:
df6 = dmatrix('C(State)  +  Wage_Rate_From_1 + Prevailing_Wage_1_x + C(Occupation_Code) + Nbr_Immigrants', df5, return_type = 'dataframe')

In [ ]:
df6.head()

In [ ]:
scale = StandardScaler()

In [ ]:
X_scale = scale.fit(df6).transform(df6)

In [ ]:
km = KMeans(n_clusters=10, n_init=20)
km.fit(X_scale)
#let's segment the group into 10 different clusters

In [ ]:
print km.inertia_

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
columns = {str(x): scale.inverse_transform(km.cluster_centers_[x]) for x in range(0,10)}
pd.DataFrame(columns, index=df6.columns)

In [ ]:
df6['segments'] = km.labels_

In [ ]:
df6.head()

In [ ]:
df6.segments.value_counts()
# we probably should make K smaller let's do 6

In [ ]:
df7=df6.drop(['segments'], 1)

In [ ]:
kmv2 = KMeans(n_clusters=6, n_init=10)
kmv2.fit(X_scale)
#let's segment the group into 6 different clusters this time changing the parameters

In [ ]:
print kmv2.inertia_

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
columns = {str(x): scale.inverse_transform(kmv2.cluster_centers_[x]) for x in range(0,6)}
pd.DataFrame(columns, index=df7.columns)

In [ ]:
df7['segments'] =kmv2.labels_

In [ ]:
df7['segments'].value_counts()
# still seeing a big gap in the data let's try 4 clusters

In [ ]:
df7= df7.drop(['segments'],1)

In [ ]:
df8= df7

In [ ]:
kmv3 = KMeans(n_clusters=4, n_init=30)
kmv3.fit(X_scale)

In [ ]:
print kmv3.inertia_

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
columns = {str(x): scale.inverse_transform(kmv3.cluster_centers_[x]) for x in range(0,4)}
pd.DataFrame(columns, index=df8.columns)

In [ ]:
df8['segments'] =kmv3.labels_

In [ ]:
df8['segments'].value_counts()

In [178]:
df8[df8['segments']==0].sample(5)

,Intercept,C(State)[T.AL],C(State)[T.AR],C(State)[T.AS],C(State)[T.AZ],C(State)[T.CA],C(State)[T.CO],C(State)[T.CT],C(State)[T.DC],C(State)[T.DE],...,C(Occupation_Code)[T.914],C(Occupation_Code)[T.924],C(Occupation_Code)[T.962],C(Occupation_Code)[T.976],C(Occupation_Code)[T.979],C(Occupation_Code)[T.989],Wage_Rate_From_1,Prevailing_Wage_1_x,Nbr_Immigrants,segments
94103,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,53000.000,52541.000,1.000,0
152392,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,100000.000,87402.000,1.000,0
36098,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,70000.000,56992.000,1.000,0
93652,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,85280.000,85280.000,1.000,0
125496,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,65000.000,46134.000,1.000,0


In [ ]:
# hmm looks like our cluster is broken out into two segments lets check to see if the company affects the segmentation